# Breaking the Transformer Bottleneck

## Preparing Data

In [4]:
# Importing relevant packages
import io
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

## Importing custom files
from data import wikitext

## Establishing devices
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Setting up vocabulary
train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
counter = Counter()
for line in train_iter:
    counter.update(tokenizer(line))
vocab = Vocab(counter)

wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:02<00:00, 2.20MB/s]


In [5]:
# Splitting data
train_iter, val_iter, test_iter = WikiText2()
train_data = wikitext.data_process(train_iter, vocab, tokenizer)
val_data = wikitext.data_process(val_iter, vocab, tokenizer)
test_data = wikitext.data_process(test_iter, vocab, tokenizer)

## Batch data
batch_size = 20
eval_batch_size = 10
train_data = wikitext.batchify(train_data, batch_size, device)
val_data = wikitext.batchify(val_data, eval_batch_size, device)
test_data = wikitext.batchify(test_data, eval_batch_size, device)

chunk_length = 35